In [ ]:
!nvidia-smi

Fri Jan 26 01:21:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


## Install Grounding DINO and Segment Anything Model

Our project will use two groundbreaking designs - [Grounding DINO](https://github.com/IDEA-Research/GroundingDINO) - for zero-shot detection and [Segment Anything Model (SAM)](https://github.com/facebookresearch/segment-anything) - for converting boxes into segmentations. We have to install them first.


In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!git checkout -q 57535c5a79791cb76e36fdb64975271354f10251
!pip install -q -e .

/content
Cloning into 'GroundingDINO'...
remote: Enumerating objects: 421, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 421 (delta 143), reused 127 (delta 127), pack-reused 233
Receiving objects: 100% (421/421), 12.85 MiB | 13.94 MiB/s, done.
Resolving deltas: 100% (215/215), done.
/content/GroundingDINO
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.8 MB/s eta 0:00:00


In [ ]:
%cd {HOME}

import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/content
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-hfq4ixao
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-hfq4ixao
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36587 sha256=9fa518293ec8af8395f213ba3e61f3dbe44de2b1eac5010e38d1ceb7b6bdae24
  Stored in directory: /tmp/pip-ephem-wheel-cache-ydtkbn_s/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


**NOTE:** To glue all the elements of our demo together we will use the [`supervision`](https://github.com/roboflow/supervision) pip package, which will help us **process, filter and visualize our detections as well as to save our dataset**. A lower version of the `supervision` was installed with Grounding DINO. However, in this demo we need the functionality introduced in the latest versions. Therefore, we uninstall the current `supervsion` version and install version `0.6.0`.



In [ ]:
!pip uninstall -y supervision
!pip install -q supervision==0.6.0

import supervision as sv
print(sv.__version__)

Found existing installation: supervision 0.4.0
Uninstalling supervision-0.4.0:
  Successfully uninstalled supervision-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
groundingdino 0.1.0 requires supervision==0.4.0, but you have supervision 0.6.0 which is incompatible.
0.6.0


**NOTE:** At the end of the tutorial we will upload our annotations to [Roboflow](roboflow.com). To automate this process with the API, let's install the `roboflow` pip package.

In [ ]:
!pip install -q roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
groundingdino 0.1.0 requires supervision==0.4.0, but you have supervision 0.6.0 which is incompatible.


### Download Grounding DINO Model Weights

To run Grounding DINO we need two files - configuration and model weights. The configuration file is part of the [Grounding DINO](https://github.com/IDEA-Research/GroundingDINO) repository, which we have already cloned. The weights file, on the other hand, we need to download. We write the paths to both files to the `GROUNDING_DINO_CONFIG_PATH` and `GROUNDING_DINO_CHECKPOINT_PATH` variables and verify if the paths are correct and the files exist on disk.

In [ ]:
import os

GROUNDING_DINO_CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(GROUNDING_DINO_CONFIG_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CONFIG_PATH))

/content/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True


In [ ]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

/content
/content/weights


In [ ]:
import os

GROUNDING_DINO_CHECKPOINT_PATH = os.path.join(HOME, "weights", "groundingdino_swint_ogc.pth")
print(GROUNDING_DINO_CHECKPOINT_PATH, "; exist:", os.path.isfile(GROUNDING_DINO_CHECKPOINT_PATH))

/content/weights/groundingdino_swint_ogc.pth ; exist: True


### Download Segment Anything Model (SAM) Weights

As with Grounding DINO, in order to run SAM we need a weights file, which we must first download. We write the path to local weight file to `SAM_CHECKPOINT_PATH` variable and verify if the path is correct and the file exist on disk.

In [ ]:
%cd {HOME}
!mkdir -p {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

/content
/content/weights


In [ ]:
import os

SAM_CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(SAM_CHECKPOINT_PATH, "; exist:", os.path.isfile(SAM_CHECKPOINT_PATH))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Load Grounding DINO Model

In [ ]:
%cd {HOME}/GroundingDINO

from groundingdino.util.inference import Model

grounding_dino_model = Model(model_config_path=GROUNDING_DINO_CONFIG_PATH, model_checkpoint_path=GROUNDING_DINO_CHECKPOINT_PATH)

/content/GroundingDINO


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

### Load Segment Anything Model (SAM)

In [ ]:
SAM_ENCODER_VERSION = "vit_h"

In [ ]:
from segment_anything import sam_model_registry, SamPredictor

sam = sam_model_registry[SAM_ENCODER_VERSION](checkpoint=SAM_CHECKPOINT_PATH).to(device=DEVICE)
sam_predictor = SamPredictor(sam)

### Prompting SAM with detected boxes

In [ ]:
import numpy as np
from segment_anything import SamPredictor


def segment(sam_predictor: SamPredictor, image: np.ndarray, xyxy: np.ndarray) -> np.ndarray:
    sam_predictor.set_image(image)
    result_masks = []
    for box in xyxy:
        masks, scores, logits = sam_predictor.predict(
            box=box,
            multimask_output=True
        )
        index = np.argmax(scores)
        result_masks.append(masks[index])
    return np.array(result_masks)

In [ ]:
import cv2

# convert detections to masks
detections.mask = segment(
    sam_predictor=sam_predictor,
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
    xyxy=detections.xyxy
)

# annotate image with detections
box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()
labels = [
    f"{CLASSES[class_id]} {confidence:0.2f}"
    for _, _, confidence, class_id, _
    in detections]
annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

%matplotlib inline
sv.plot_image(annotated_image, (16, 16))

NameError: name 'image' is not defined

## Full Dataset Mask Auto Annotation

In [ ]:
os.makedirs("/content/Mascaras", exist_ok=True)
os.makedirs("/content/Tortugas", exist_ok=True)
os.makedirs("/content/outputSobel/", exist_ok=True)


### Extract labels from images

In [ ]:
import os

def listar_directorios(directory):
    return [os.path.join(directory, d) for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]

directorios = listar_directorios("/content/drive/MyDrive/Tortugas SC")

# Imprimir la lista de directorios
print("Directorios encontrados:")
for directorio in directorios:
    print(directorio)


Directorios encontrados:
/content/drive/MyDrive/Tortugas SC/1
/content/drive/MyDrive/Tortugas SC/8
/content/drive/MyDrive/Tortugas SC/6
/content/drive/MyDrive/Tortugas SC/9
/content/drive/MyDrive/Tortugas SC/5
/content/drive/MyDrive/Tortugas SC/2
/content/drive/MyDrive/Tortugas SC/7
/content/drive/MyDrive/Tortugas SC/3
/content/drive/MyDrive/Tortugas SC/10
/content/drive/MyDrive/Tortugas SC/4


In [ ]:
import shutil

# Ruta de la carpeta que deseas eliminar
ruta_carpeta = '/content/Tortugas'

# Elimina la carpeta y su contenido
shutil.rmtree(ruta_carpeta)

# Ruta de la carpeta que deseas eliminar
ruta_carpeta = '/content/Mascaras'

# Elimina la carpeta y su contenido
shutil.rmtree(ruta_carpeta)


In [ ]:
import os

IMAGES_EXTENSIONS = ['jpg', 'jpeg', 'png']

CLASSES = ['turtle', 'head', 'eye', 'shell', 'back fin', 'fin']
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

In [ ]:
i=0
j=2
k=1
z=1

In [ ]:
from typing import List

def enhance_class_name(class_names: List[str]) -> List[str]:
    return [
        f"all {class_name}s"
        for class_name
        in class_names
    ]

In [ ]:
import cv2
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

for directorio in tqdm(directorios):

    if i>=j:
        j=j+1

        ruta,carpeta = os.path.split(directorio)
        print(carpeta)
        ruta_carpeta_mascaras_tortuga = os.path.join("/content/Mascaras", carpeta)
        ruta_tortugas = os.path.join("/content/Tortugas", carpeta)
        ruta_sobel = os.path.join("/content/outputSobel", carpeta)
        # Crea el directorio si no existe
        os.makedirs(ruta_carpeta_mascaras_tortuga, exist_ok=True)
        os.makedirs(ruta_tortugas, exist_ok=True)
        os.makedirs(ruta_sobel, exist_ok=True)
        images = {}
        annotations = {}
        IMAGES_DIRECTORY = directorio
        image_paths = sv.list_files_with_extensions(
            directory=IMAGES_DIRECTORY,
            extensions=IMAGES_EXTENSIONS)

        for image_path in tqdm(image_paths):
          if z>=k:
            k = k+1
            print(image_path)
            image_name = image_path.name
            image_path = str(image_path)
            image = cv2.imread(image_path)

            detections = grounding_dino_model.predict_with_classes(
                image=image,
                classes=enhance_class_name(class_names=CLASSES),
                box_threshold=BOX_TRESHOLD,
                text_threshold=TEXT_TRESHOLD
            )
            detections = detections[detections.class_id != None]
            detections.mask = segment(
                sam_predictor=sam_predictor,
                image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
                xyxy=detections.xyxy
            )
            #grid_size_dimension = math.ceil(math.sqrt(len(detections.mask)))
            labels = [
                      f"{CLASSES[class_id]} {confidence:0.2f}"
                      for _, _, confidence, class_id, _
                      in detections]
            titles1 = [
                CLASSES[class_id]
                for class_id
                in detections.class_id
            ]
            titles = [
                f"{CLASSES[detections.class_id[i]]} {detections.confidence[i]:0.2f}"
                for i in range(len(detections.class_id))
            ]


            for title, mask in zip(titles, detections.mask):
                image = cv2.imread(image_path)
                # Obtener la última carpeta y el nombre del archivo
                nombre_img = os.path.splitext(os.path.basename(image_path))[0]
                ruta_mascara = "/content/Mascaras/"+carpeta+"/"+nombre_img
                print(ruta_mascara)
                os.makedirs(ruta_mascara , exist_ok=True)
                ruta_imgMascara = os.path.join(ruta_mascara, title + ".jpeg")

               # plt.imshow(mask)
               # plt.show()
                plt.savefig(ruta_mascara+"/"+title+".jpeg", bbox_inches='tight', pad_inches=0)

                mask1 = cv2.imread(ruta_imgMascara)
                image[np.where(mask == 0)] = [255, 255, 255]

                rutaImgSegmentada = '/content/Tortugas/'+carpeta+'/'+nombre_img
                os.makedirs(rutaImgSegmentada, exist_ok=True)

                print(rutaImgSegmentada)
                denoised_image = cv2.medianBlur(image, 7)
                cv2.imwrite(rutaImgSegmentada  +"/"+title+'.jpeg', denoised_image)





            images[image_name] = image
            annotations[image_name] = detections
          z=z+1
    i=i+1

  0%|          | 0/10 [00:00<?, ?it/s]

8


  0%|          | 0/75 [00:00<?, ?it/s]

/content/drive/MyDrive/Tortugas SC/8/20231206_071120.jpeg


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


...

/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/Mascaras/8/20231206_071140
/content/Tortugas/8/20231206_071140
/content/drive/MyDrive/Tortugas SC/8/20231206_071143.jpeg


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


/content/Mascaras/8/20231206_071143
/content/Tortugas/8/20231206_071143


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
import os

import locale
locale.getpreferredencoding = lambda: "UTF-8"



In [ ]:
%cd {HOME}

#! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision
!pip install transformers


/content
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-a45vgj6g
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-a45vgj6g
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 819.0 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=ecdc7ede59480896f7301a6c78e970678ab6568b85762d143b5541d6c23622be
  Stored in directory: /tmp/pip-ephem-wheel-cache-q_dj6hym/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


Clip

In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
import os

def obtener_rutas_y_ultimas_dos_carpetas(ruta_carpeta):
    rutas_archivos = []
    ultimas_dos_carpetas = []

    # Recorre la carpeta y sus subcarpetas
    for carpeta_actual, subcarpetas, archivos in os.walk(ruta_carpeta):
        for archivo in archivos:
            # Construye la ruta completa del archivo
            ruta_completa = os.path.join(carpeta_actual, archivo)

            # Obtiene las últimas dos carpetas en la ruta
            carpetas = carpeta_actual.split(os.path.sep)[-2:]

            # Agrega la ruta a la lista de rutas de archivos
            rutas_archivos.append(ruta_completa)

            # Agrega las últimas dos carpetas a la lista correspondiente
            ultimas_dos_carpetas.append(carpetas)

    return rutas_archivos, ultimas_dos_carpetas

# Ruta de la carpeta principal
ruta_principal = '/content/Tortugas'


rutas_archivos, ultimas_dos_carpetas = obtener_rutas_y_ultimas_dos_carpetas(ruta_principal)



In [ ]:
def es_adjetivo(palabra):
    return palabra.lower() in adjetivos

def calcular_sobel(image, thresh_min=20, thresh_max=100):
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Aplicar Sobel en x e y
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)

    # Calcular el gradiente y su magnitud
    gradient = np.sqrt(sobelx ** 2 + sobely ** 2)
    abs_gradient = np.absolute(gradient)
    scaled_sobel = np.uint8(255 * abs_gradient / np.max(abs_gradient))

    # Aplicar umbral para obtener la imagen binaria
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    return sxbinary



def porcentaje_pixeles_blancos(img):
    try:

        # Convertir la imagen a un array de NumPy

        img_array = np.array(img)

        # Contar los píxeles blancos
        píxeles_blancos = np.sum(img_array == 255)

        # Calcular el porcentaje de píxeles blancos
        porcentaje_blancos = (píxeles_blancos / img_array.size) * 100

        return porcentaje_blancos

    except Exception as e:
        print(f"Error al procesar la imagen {imagen}: {str(e)}")
        return None

def procesar_imagen(ruta_imagen, ruta_carpetasClip, ruta_carpetasSobel,ruta_carpetasTortugasRef, nombre_imagen):
    imagen = cv2.imread(ruta_imagen)
    # Utilizar CLIP para clasificar las imágenes segmentadas
    if nombre_imagen != 'eye':
        text = [

            ' top view of a sea turtle''s head',
            ' top view of a sea turtle''s shell',
            ' side left view of a sea turtle''s head',
            ' side left view of a sea turtle''s shell',
            ' side right view of a sea turtle''s head',
            ' side right view of a sea turtle''s shell',
            ' side right view of a sea turtle''s fin',
            ' side left view of a sea turtle''s fin',
            'fin',
            'turtle''s shell',
            'turtle''s head',
            'front fin',
            'back fin',
            'tail'



        ]

        inputs = processor(text, images=imagen, return_tensors="pt", padding=True)
        outputs = model(**inputs)
        logits_per_image = outputs.logits_per_image
        probs = logits_per_image.softmax(dim=1)

        # Obtener la probabilidad más alta y su índice
        highest_prob, highest_prob_index = probs.max(dim=1)

        # Agregar el resultado a la lista
        results.append({"path": image,  "prob": highest_prob.item(), "text": text[highest_prob_index.item()]})
        # Redondear la probabilidad a dos decimales
        rounded_prob = round(highest_prob.item(), 2)

        # Convertir la probabilidad redondeada a una cadena de texto
        rounded_prob_str = "{:.2f}".format(rounded_prob)

        porcentajeBlanco = porcentaje_pixeles_blancos(imagen)


        if rounded_prob  > 0.2 and porcentajeBlanco < 95 :
                    adjetivos = [
                        "fin",
                        "shell",
                        "head",
                        "back fin",
                        "tail",
                        "left",
                        "right",
                        "top",
                        "turtle"
                    ]

                    # Dividir el texto en palabras
                    texto = text[highest_prob_index.item()]
                    palabras = texto.split()

                    # Filtrar las palabras, manteniendo solo aquellas que son adjetivos
                    palabras_filtradas = [palabra for palabra in palabras if es_adjetivo(palabra)]

                    # Reconstruir el texto a partir de las palabras filtradas
                    texto_filtrado = ' '.join(palabras_filtradas)


               # if 'turtle' in text[highest_prob_index.item()]:

                    grad_binary = cv2.medianBlur(imagen, 7)

                    # SOBEL

                    binary_image = calcular_sobel(denoised_image)

                    rutaImgClip = ruta_carpetasClip + "/D " + nombre_imagen + " C " + texto_filtrado + " " + rounded_prob_str + '.jpeg'
                    cv2.imwrite(rutaImgClip, imagen)

                    rutaImgRef = ruta_carpetasTortugasRef+ "/D " + nombre_imagen + " C " + texto_filtrado + " " + rounded_prob_str + '.jpeg'
                    cv2.imwrite(rutaImgRef, imagen)

                    rutaImgSobel = ruta_carpetasSobel +"/D " + nombre_imagen+ " C " + texto_filtrado + " " + rounded_prob_str + '.jpeg'
                    cv2.imwrite(rutaImgSobel, grad_binary * 255)


In [ ]:
        adjetivos = [

            "fin",
            "shell",
            "head",
            "back fin",
            "Tail",
            "left",
            "right",
            "top",
            "turtle"
        ]

In [ ]:
results = []

ruta_baseClip= "/content/Tortugas_Clip_Clasificacion"
ruta_baseSobel= "/content/outputSobel"
ruta_baseTortugasRef= "/content/Tortugas_Referencia"
carpetasClip = []
os.makedirs(ruta_baseClip, exist_ok=True)
os.makedirs(ruta_baseSobel, exist_ok=True)
os.makedirs(ruta_baseTortugasRef, exist_ok=True)
for carpetas, ruta_imagen in zip(ultimas_dos_carpetas, rutas_archivos):

        ruta_carpetasClip = os.path.join(ruta_baseClip,*carpetas)
        ruta_carpetasSobel = os.path.join(ruta_baseSobel,*carpetas)
         # Obtener la antepenúltima carpeta
        antepenultima_carpeta = carpetas[-2]

        # Unir la ruta solo con la antepenúltima carpeta
        ruta_carpetasTortugasRef = os.path.join(ruta_baseTortugasRef, antepenultima_carpeta)

        nombre_imagen = os.path.splitext(os.path.basename(ruta_imagen))[0]
        carpetasClip.append(ruta_carpetasClip)

        # Crea la carpeta si no existe
        os.makedirs(ruta_carpetasClip, exist_ok=True)
        os.makedirs(ruta_carpetasSobel, exist_ok=True)
        os.makedirs(ruta_carpetasTortugasRef, exist_ok=True)
        #print(ruta_carpetasClip)

        procesar_imagen(ruta_imagen,ruta_carpetasClip,ruta_carpetasSobel,ruta_carpetasTortugasRef,nombre_imagen)




Descripción por imagen (carpetas)

In [ ]:
import os
from collections import Counter

def agregar_adjetivos_y_cambiar_nombre(carpeta_segmentaciones):
    # Obtener el nombre de la carpeta actual
    nombre_carpeta = os.path.basename(carpeta_segmentaciones)

    # Inicializar el contador
    contador_adjetivos = Counter()

    # Iterar sobre los archivos en la carpeta
    for archivo in os.listdir(carpeta_segmentaciones):
        # Obtener el nombre del archivo sin la extensión
        nombre_archivo, _ = os.path.splitext(archivo)

        # Lista de adjetivos
        adjetivos = [

            "fin",
            "shell",
            "head",
            "back fin",
            "Tail",
            "left",
            "right",
            "top",
            "turtle"
        ]

        # Contar la frecuencia de cada adjetivo en el nombre del archivo
        for adjetivo in adjetivos:
            if adjetivo in nombre_archivo:
                contador_adjetivos[adjetivo] += 1

    # Calcular la probabilidad de cada adjetivo
    total_adj = sum(contador_adjetivos.values())
    probabilidades = {adjetivo: frecuencia / total_adj for adjetivo, frecuencia in contador_adjetivos.items()}
    tres_adjetivos_mas_probables = [adjetivo for adjetivo, _ in contador_adjetivos.most_common(5)]

    # Agregar los adjetivos al nombre de la carpeta

    #nuevo_nombre_carpeta = nombre_carpeta + " " + " ".join([f"{adjetivo} {probabilidades[adjetivo]:.2f}" for adjetivo in tres_adjetivos_mas_probables])
    # Agregar los adjetivos al nombre de la carpeta junto con sus probabilidades
    nuevo_nombre_carpeta = nombre_carpeta + " " + " ".join([f"{adjetivo} {probabilidades.get(adjetivo, 0):.2f}" for adjetivo in contador_adjetivos.keys()])

    # Cambiar el nombre de la carpeta
    ruta_nueva_carpeta = os.path.join(os.path.dirname(carpeta_segmentaciones), nuevo_nombre_carpeta)
    os.rename(carpeta_segmentaciones, ruta_nueva_carpeta)


directorio_principal = '/content/Tortugas_Clip_Clasificacion/'
for carpeta in os.listdir(directorio_principal):

        ruta = os.path.join(directorio_principal, carpeta)
        carpetasClip = [os.path.join(ruta, nombre) for nombre in os.listdir(ruta) if os.path.isdir(os.path.join(ruta, nombre))]

        for carpeta in carpetasClip:

            agregar_adjetivos_y_cambiar_nombre(carpeta)


Comparación con imagenes de referencia

PIDINET

In [ ]:
!git clone https://github.com/hellozhuo/pidinet.git

Cloning into 'pidinet'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 553 (delta 0), reused 2 (delta 0), pack-reused 547
Receiving objects: 100% (553/553), 48.80 MiB | 38.26 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
import os
import re

# Lista de adjetivos
adjetivos = [
    "fin",
    "shell",
    "head",
    "back fin",
    "Tail",
    "turtle"
]

# Función para extraer los adjetivos del título de la imagen
def extract_adjectives(title):
    adjectives_found = []
    for adj in adjetivos:
        if adj in title:
            adjectives_found.append(adj)
    return adjectives_found



In [ ]:
import os
import re


os.makedirs("/content/outputPidinet", exist_ok=True)
# Función para extraer los adjetivos del título de la imagen
def extract_adjectives(title):
    adjectives_found = []
    for adj in adjetivos:
        if adj in title:
            adjectives_found.append(adj)
    return adjectives_found

# Función para extraer la probabilidad de una clasificación específica del título de la imagen
def extract_probability(title, classification):
    match = re.search(rf'{classification} (\d+\.\d+)', title)
    if match:
        return float(match.group(1))
    else:
        return 0.0

# Función para calcular la puntuación de calidad de una imagen basada en la probabilidad de una clasificación específica
def calculate_quality_score(probability):
    return probability

def clean_filename(filename):
    return re.sub(r'[^a-zA-Z\s]', '', filename)


# imagen de referencia
imagenOrg = "/content/Tortugas_Clip_Clasificacion/8/20231206_071130/D head 0.44 C left shell 0.41.jpeg"
adjectives = extract_adjectives(imagenOrg)

# Directorio donde se encuentran las imágenes de referencia
directory = '/content/Tortugas_Referencia/8'
output_directory = '/content/Tortugas_Temp/8'
os.makedirs(output_directory, exist_ok=True)
# Iterar sobre cada adjetivo de la imagen de referencia
for adj in adjectives:
    print(f"Evaluación para el adjetivo: {adj}")

    # Lista para almacenar las rutas de las imágenes junto con sus puntuaciones de calidad
    image_scores = []

    # Iterar sobre los archivos en el directorio
    for filename in os.listdir(directory):
        if filename.endswith('.jpeg'):
            full_path = os.path.join(directory, filename)
            probability = extract_probability(filename, adj)
            quality_score = calculate_quality_score(probability)
            image_scores.append((full_path, quality_score))

    # Ordenar las imágenes según su puntuación de calidad
    sorted_images = sorted(image_scores, key=lambda x: x[1], reverse=True)

    # Mostrar las imágenes ordenadas por calidad
    for image, score in sorted_images:
      if score > 0.4 :
            print(f'Imagen: {image}, Puntuación de calidad: {score}')
            cleaned_filename = clean_filename(os.path.basename(image))
            new_path = os.path.join(output_directory, cleaned_filename + '.jpeg')
            shutil.copy(image, new_path)

   # !python /content/pidinet/main.py --model pidinet_converted --config carv4 --sa --dil -j 4 --gpu 0 --savedir /content/outputPidinet --datadir {output_directory} --dataset Custom --evaluate /content/pidinet/trained_models/table5_pidinet.pth --evaluate-converted


Evaluación para el adjetivo: shell
Imagen: /content/Tortugas_Referencia/8/D turtle 0.48 C right shell 0.43.jpeg, Puntuación de calidad: 0.43
Imagen: /content/Tortugas_Referencia/8/D head 0.38 C right shell 0.43.jpeg, Puntuación de calidad: 0.43
Imagen: /content/Tortugas_Referencia/8/D turtle 0.65 C left shell 0.42.jpeg, Puntuación de calidad: 0.42
Imagen: /content/Tortugas_Referencia/8/D head 0.44 C left shell 0.41.jpeg, Puntuación de calidad: 0.41
Imagen: /content/Tortugas_Referencia/8/D fin 0.46 C right shell 0.41.jpeg, Puntuación de calidad: 0.41
Evaluación para el adjetivo: head
Imagen: /content/Tortugas_Referencia/8/D head 0.45 C right shell 0.37.jpeg, Puntuación de calidad: 0.45
Imagen: /content/Tortugas_Referencia/8/D head 0.44 C left shell 0.41.jpeg, Puntuación de calidad: 0.44
Imagen: /content/Tortugas_Referencia/8/D head 0.44 C fin 0.22.jpeg, Puntuación de calidad: 0.44
Imagen: /content/Tortugas_Referencia/8/D head 0.44 C left shell 0.38.jpeg, Puntuación de calidad: 0.44
Imag

In [ ]:
carpetaBase ='/content/Tortugas_Temp'
carpetasTortugas = os.listdir(carpetaBase)


for carpetaTortugas in carpetasTortugas:

    rutaCarpetaTortugas = os.path.join(carpetaBase, carpetaTortugas)
    carpetasImg = os.listdir(rutaCarpetaTortugas)
    for carpetaImagen in carpetasImg :

      rutaCarpetaImagen = os.path.join(rutaCarpetaTortugas , carpetaImagen)
      imagenes = os.listdir(rutaCarpetaImagen )
      for i,imagen in enumerate(imagenes):

                rutaImagenes = os.path.join(rutaCarpetaImagen , imagen)
                nombreImagen = os.path.splitext(os.path.basename(imagen))[0]

                ruta8img = ''
                rutaImgSobel = '/content/outputSobel/' + carpetaTortugas + '/' + carpetaImagen+ '/' +  nombreImagen + '.jpg'
                print(rutaImgSobel)
                rutaImgPidinet = '/content/outputPidinet/' + carpetaTortugas + '/'  + carpetaImagen + '/eval_results/imgs_epoch_019/' +  nombreImagen + '.jpg'
                ruta5ImgPidinet = '/content/Pidinet5/' + carpetaTortugas + '/'  + carpetaImagen + '/'
                ruta8img = '/content/8Img/' + carpetaTortugas + '/' + carpetaImagen+'/'
                ruta8imgRef = '/content/8Img_Ref/' + carpetaTortugas + '/' + carpetaImagen+'/'
                os.makedirs(ruta8img, exist_ok=True)  # Crea el directorio si no existe
                os.makedirs(ruta8imgRef, exist_ok=True)  # Crea el directorio si no existe
              #  print(ruta5ImgPidinet)
                imagenSobel = cv2.imread(rutaImgSobel)
                # imagenSobel_invertida = cv2.bitwise_not(imagenSobel)
                imagenPidinet = cv2.imread(rutaImgPidinet)
                imagenPidinet_invertida = cv2.bitwise_not(imagenPidinet)


                # Verifica que la imagenSobel no sea None
                if imagenSobel is not None and imagenPidinet is not None and imagenPidinet_invertida is not None:

                    # Obtiene las dimensiones de la imagenSobel y recorta a la imagen pobtenida por la red PIDINET
                    alto, ancho = imagenSobel.shape[0], imagenSobel.shape[1]
                    imagenPidinetRecortada1 = imagenPidinet_invertida[:alto, :ancho]
                    imagenPidinetRecortada2 = imagenPidinet_invertida[:alto, ancho : ancho*2]
                    imagenPidinetRecortada3 = imagenPidinet_invertida[:alto, ancho*2 : ancho*3]
                    imagenPidinetRecortada4 = imagenPidinet_invertida[:alto, ancho*3 : ancho*4]
                    imagenPidinetRecortada5 = imagenPidinet_invertida[:alto, ancho*4 : ancho*5]

                    !mkdir {ruta5ImgPidinet}

                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'1.jpg', imagenPidinetRecortada1)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'2.jpg', imagenPidinetRecortada2)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'3.jpg', imagenPidinetRecortada3)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'4.jpg', imagenPidinetRecortada4)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'5.jpg', imagenPidinetRecortada5)


                    cv2.imwrite(ruta8img + nombreImagen +'1.jpg', imagenPidinetRecortada1)
                    cv2.imwrite(ruta8img + nombreImagen +'2.jpg', imagenPidinetRecortada2)
                    cv2.imwrite(ruta8img + nombreImagen +'3.jpg', imagenPidinetRecortada3)
                    cv2.imwrite(ruta8img + nombreImagen +'4.jpg', imagenPidinetRecortada4)
                    cv2.imwrite(ruta8img + nombreImagen +'5.jpg', imagenPidinetRecortada5)

                    if  imagenSobel.shape == imagenPidinetRecortada1.shape:
                            # Suma las dos imágenes
                            resultado1 = cv2.add( imagenSobel, imagenPidinetRecortada1)
                            resultado2 = cv2.add( imagenSobel, imagenPidinetRecortada2)
                            cv2.imwrite(ruta8img + nombreImagen +'Sobel.jpg', imagenSobel)
                            # Guarda la imagen resultante
                            rutaImgSobelPidinet = '/content/outputSobelPidinet/' + carpetaImagen + '/'
                            !mkdir {rutaImgSobelPidinet}
                            cv2.imwrite(rutaImgSobelPidinet + nombreImagen +'_Pidinet1.jpg', resultado1)
                            cv2.imwrite(rutaImgSobelPidinet + nombreImagen +'_Pidinet2.jpg', resultado2)
                            cv2.imwrite(ruta8img + nombreImagen +'_Pidinet1.jpg', resultado1)
                            cv2.imwrite(ruta8img + nombreImagen +'_Pidinet2.jpg', resultado2)
                    else:
                          print("Las dimensiones de las imágenes no coinciden.")
                          print(imagenPidinetRecortada1.shape)



                    #si en el nombre de la imagen estan las palabras (top, left, right)
                    if any(palabra_clave in nombreImagen for palabra_clave in ["top", "left", "right"]):

                        cv2.imwrite(ruta8imgRef + nombreImagen +'1.jpg', imagenPidinetRecortada1)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'2.jpg', imagenPidinetRecortada2)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'3.jpg', imagenPidinetRecortada3)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'4.jpg', imagenPidinetRecortada4)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'5.jpg', imagenPidinetRecortada5)


                        if  imagenSobel.shape == imagenPidinetRecortada1.shape:
                            # Suma las dos imágenes
                            resultado1 = cv2.add( imagenSobel, imagenPidinetRecortada1)
                            resultado2 = cv2.add( imagenSobel, imagenPidinetRecortada2)
                            cv2.imwrite(ruta8imgRef+ nombreImagen +'Sobel.jpg', imagenSobel)
                            # Guarda la imagen resultante
                            cv2.imwrite(ruta8imgRef + nombreImagen +'_Pidinet1.jpg', resultado1)
                            cv2.imwrite(ruta8imgRef + nombreImagen +'_Pidinet2.jpg', resultado2)
                        else:
                            print("Las dimensiones de las imágenes no coinciden.")
                            print(imagenPidinetRecortada1.shape)

                # Guarda la imagen recortada

                #/content/outputSobel/4/10N_a photo of a turtle.jpeg
                # /content/outputPidinet/4/eval_results/imgs_epoch_019/10N_a photo of a turtle.jpg

NotADirectoryError: [Errno 20] Not a directory: '/content/Tortugas_Temp/8/D turtle  C left shell jpeg.jpeg'

ORB

In [ ]:
carpeta_imagenes = "/content/Tortugas_Temp"
imagen1 = cv2.imread(imagenOrg )
archivos_en_carpeta = os.listdir(carpeta_imagenes)

# Filtrar solo los archivos de imagen
imagenes_con_direccion = [os.path.join(carpeta_imagenes, archivo) for archivo in archivos_en_carpeta if archivo.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

# Mostrar las direcciones de las imágenes
for imagen_2 in imagenes_con_direccion:


                                      imagen2 = cv2.imread(imagen_2)
                                      print(f"Coincidencias encontradas en palabras clave para imágenes:")

                                      print(f"Palabras clave coincidentes: {coincidencias}")
                                      print(imagen)
                                      print(imagen_2)

                                      # Definir los parámetros del ORB
                                      nfeatures = 10000  # Número máximo de características a retener
                                      scaleFactor = 1.1  # Factor de escala entre niveles de pirámide
                                      nlevels = 8  # Número de niveles en la pirámide
                                      edgeThreshold = 20  # Umbral para la supresión de bordes

                                      # Crear el objeto ORB con los parámetros especificados
                                      orb = cv2.ORB_create(nfeatures=nfeatures, scaleFactor=scaleFactor, nlevels=nlevels, edgeThreshold=edgeThreshold)

                                      keypoints1, descriptors1 = orb.detectAndCompute(imagen1, None)
                                      keypoints2, descriptors2 = orb.detectAndCompute(imagen2, None)

                                      img_orb1 = cv2.drawKeypoints(imagen1, keypoints1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                                      img_orb2 = cv2.drawKeypoints(imagen2, keypoints2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                                      img_orb1 = cv2.resize(img_orb1, (img_orb2.shape[1], img_orb2.shape[0]))

                                      frame = cv2.hconcat((img_orb1,img_orb2))


                                      bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                                      matches = bf.match(descriptors1, descriptors2)
                                      matches = sorted(matches, key=lambda x: x.distance)

                                      img_matches = cv2.drawMatches(imagen1, keypoints1, imagen2, keypoints2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

                                      good_matches = []
                                      for m in matches:
                                          pts1 = keypoints1[m.queryIdx].pt
                                          pts2 = keypoints2[m.trainIdx].pt
                                          good_matches.append(m)

                                      pts1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                                      pts2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

                                        # Inicializar una lista para almacenar el número de coincidencias en cada iteración de RANSAC
                                      num_matches_per_iteration = []
                                      numIteraciones=500
                                      for iteration in range(numIteraciones):  # Número de iteraciones en RANSAC (ajusta según sea necesario)
                                          # Encontrar homografía usando RANSAC
                                          M, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, ransacReprojThreshold=5.0, maxIters=iteration + 1)

                                          # Obtener la máscara de inliers
                                          matchesMask = mask.ravel().tolist()

                                          # Filtrar correspondencias basadas en la máscara RANSAC
                                          filtered_matches = [good_matches[i] for i in range(len(matchesMask)) if matchesMask[i] == 1]

                                          # Almacenar el número de coincidencias en esta iteración
                                          num_matches_per_iteration.append(len(filtered_matches))

                                        #   Dibujar las correspondencias filtradas (opcional, puedes comentar esta línea si no quieres visualizar cada iteración)
                                        #  img_matches = cv2.drawMatches(imagen1, keypoints1, imagen2, keypoints2, filtered_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                                        #  plt.figure(figsize=(10, 5))
                                        #  plt.imshow(img_matches)
                                        #  plt.title(f'Correspondencias Filtradas con Homografía RANSAC - Iteración {iteration + 1}')
                                        #  plt.show()
                                      porcentaje_aciertos = (len(filtered_matches) / min(len(keypoints1), len(keypoints2))) * 100
                                      # Dibujar las correspondencias filtradas
                                      img_matches = cv2.drawMatches(imagen1, keypoints1, imagen2, keypoints2, filtered_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                                      # Dibujar las mejores coincidencias en la imagen
                                      cv2_imshow(img_matches)
                                      import matplotlib.pyplot as plt

                                      # Graficar el número de coincidencias en cada iteración de RANSAC
                                      plt.figure(figsize=(10, 5))
                                      plt.plot(range(1, numIteraciones+1), num_matches_per_iteration, marker='o')
                                      plt.title('Número de Coincidencias en Cada Iteración de RANSAC')
                                      plt.xlabel('Iteración de RANSAC')
                                      plt.ylabel('Número de Coincidencias')
                                      plt.grid(True)
                                      plt.show()

                                      print(f"Porcentaje de coincidencia: {porcentaje_aciertos:.2f}%")

In [ ]:
import shutil

# Ruta de la carpeta que deseas eliminar
ruta_carpeta = '/content/Tortugas_Clip_Clasificacion'
# Elimina la carpeta y su contenido
shutil.rmtree(ruta_carpeta)
ruta_carpeta = '/content/outputSobel'
# Elimina la carpeta y su contenido
shutil.rmtree(ruta_carpeta)
ruta_carpeta = '/content/outputPidinet'
# Elimina la carpeta y su contenido
shutil.rmtree(ruta_carpeta)


Concatenar Imagenes


In [ ]:
!mkdir {HOME}/outputSobelPidinet
!mkdir {HOME}/Pidinet5/
!mkdir {HOME}/8Img/
!mkdir {HOME}/8Img_Ref/

# Ruta de la carpeta
carpeta = '/content/Tortugas'

# Obtener la lista de elementos (archivos y subcarpetas) en la carpeta
elementos = os.listdir(carpeta)

for elemento in elementos:
    !mkdir {HOME}/outputSobelPidinet/{elemento}
    !mkdir {HOME}/Pidinet5/{elemento}
    !mkdir {HOME}/8Img/{elemento}
    !mkdir {HOME}/8Img_Ref/{elemento}


In [ ]:
carpetaBase ='/content/Tortugas/'
carpetasTortugas = os.listdir(carpetaBase)


for carpetaTortugas in carpetasTortugas:

    rutaCarpetaTortugas = os.path.join(carpetaBase, carpetaTortugas)
    carpetasImg = os.listdir(rutaCarpetaTortugas)
    for carpetaImagen in carpetasImg :

      rutaCarpetaImagen = os.path.join(rutaCarpetaTortugas , carpetaImagen)
      imagenes = os.listdir(rutaCarpetaImagen )
      for i,imagen in enumerate(imagenes):

                rutaImagenes = os.path.join(rutaCarpetaImagen , imagen)
                nombreImagen = os.path.splitext(os.path.basename(imagen))[0]

                ruta8img = ''
                rutaImgSobel = '/content/outputSobel/' + carpetaTortugas + '/' + carpetaImagen+ '/' +  nombreImagen + '.jpg'
                print(rutaImgSobel)
                rutaImgPidinet = '/content/outputPidinet/' + carpetaTortugas + '/'  + carpetaImagen + '/eval_results/imgs_epoch_019/' +  nombreImagen + '.jpg'
                ruta5ImgPidinet = '/content/Pidinet5/' + carpetaTortugas + '/'  + carpetaImagen + '/'
                ruta8img = '/content/8Img/' + carpetaTortugas + '/' + carpetaImagen+'/'
                ruta8imgRef = '/content/8Img_Ref/' + carpetaTortugas + '/' + carpetaImagen+'/'
                os.makedirs(ruta8img, exist_ok=True)  # Crea el directorio si no existe
                os.makedirs(ruta8imgRef, exist_ok=True)  # Crea el directorio si no existe
              #  print(ruta5ImgPidinet)
                imagenSobel = cv2.imread(rutaImgSobel)
                # imagenSobel_invertida = cv2.bitwise_not(imagenSobel)
                imagenPidinet = cv2.imread(rutaImgPidinet)
                imagenPidinet_invertida = cv2.bitwise_not(imagenPidinet)


                # Verifica que la imagenSobel no sea None
                if imagenSobel is not None and imagenPidinet is not None and imagenPidinet_invertida is not None:

                    # Obtiene las dimensiones de la imagenSobel y recorta a la imagen pobtenida por la red PIDINET
                    alto, ancho = imagenSobel.shape[0], imagenSobel.shape[1]
                    imagenPidinetRecortada1 = imagenPidinet_invertida[:alto, :ancho]
                    imagenPidinetRecortada2 = imagenPidinet_invertida[:alto, ancho : ancho*2]
                    imagenPidinetRecortada3 = imagenPidinet_invertida[:alto, ancho*2 : ancho*3]
                    imagenPidinetRecortada4 = imagenPidinet_invertida[:alto, ancho*3 : ancho*4]
                    imagenPidinetRecortada5 = imagenPidinet_invertida[:alto, ancho*4 : ancho*5]

                    !mkdir {ruta5ImgPidinet}

                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'1.jpg', imagenPidinetRecortada1)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'2.jpg', imagenPidinetRecortada2)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'3.jpg', imagenPidinetRecortada3)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'4.jpg', imagenPidinetRecortada4)
                    cv2.imwrite(ruta5ImgPidinet + nombreImagen +'5.jpg', imagenPidinetRecortada5)


                    cv2.imwrite(ruta8img + nombreImagen +'1.jpg', imagenPidinetRecortada1)
                    cv2.imwrite(ruta8img + nombreImagen +'2.jpg', imagenPidinetRecortada2)
                    cv2.imwrite(ruta8img + nombreImagen +'3.jpg', imagenPidinetRecortada3)
                    cv2.imwrite(ruta8img + nombreImagen +'4.jpg', imagenPidinetRecortada4)
                    cv2.imwrite(ruta8img + nombreImagen +'5.jpg', imagenPidinetRecortada5)

                    if  imagenSobel.shape == imagenPidinetRecortada1.shape:
                            # Suma las dos imágenes
                            resultado1 = cv2.add( imagenSobel, imagenPidinetRecortada1)
                            resultado2 = cv2.add( imagenSobel, imagenPidinetRecortada2)
                            cv2.imwrite(ruta8img + nombreImagen +'Sobel.jpg', imagenSobel)
                            # Guarda la imagen resultante
                            rutaImgSobelPidinet = '/content/outputSobelPidinet/' + carpetaImagen + '/'
                            !mkdir {rutaImgSobelPidinet}
                            cv2.imwrite(rutaImgSobelPidinet + nombreImagen +'_Pidinet1.jpg', resultado1)
                            cv2.imwrite(rutaImgSobelPidinet + nombreImagen +'_Pidinet2.jpg', resultado2)
                            cv2.imwrite(ruta8img + nombreImagen +'_Pidinet1.jpg', resultado1)
                            cv2.imwrite(ruta8img + nombreImagen +'_Pidinet2.jpg', resultado2)
                    else:
                          print("Las dimensiones de las imágenes no coinciden.")
                          print(imagenPidinetRecortada1.shape)



                    #si en el nombre de la imagen estan las palabras (top, left, right)
                    if any(palabra_clave in nombreImagen for palabra_clave in ["top", "left", "right"]):

                        cv2.imwrite(ruta8imgRef + nombreImagen +'1.jpg', imagenPidinetRecortada1)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'2.jpg', imagenPidinetRecortada2)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'3.jpg', imagenPidinetRecortada3)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'4.jpg', imagenPidinetRecortada4)
                        cv2.imwrite(ruta8imgRef + nombreImagen +'5.jpg', imagenPidinetRecortada5)


                        if  imagenSobel.shape == imagenPidinetRecortada1.shape:
                            # Suma las dos imágenes
                            resultado1 = cv2.add( imagenSobel, imagenPidinetRecortada1)
                            resultado2 = cv2.add( imagenSobel, imagenPidinetRecortada2)
                            cv2.imwrite(ruta8imgRef+ nombreImagen +'Sobel.jpg', imagenSobel)
                            # Guarda la imagen resultante
                            cv2.imwrite(ruta8imgRef + nombreImagen +'_Pidinet1.jpg', resultado1)
                            cv2.imwrite(ruta8imgRef + nombreImagen +'_Pidinet2.jpg', resultado2)
                        else:
                            print("Las dimensiones de las imágenes no coinciden.")
                            print(imagenPidinetRecortada1.shape)

                # Guarda la imagen recortada

                #/content/outputSobel/4/10N_a photo of a turtle.jpeg
                # /content/outputPidinet/4/eval_results/imgs_epoch_019/10N_a photo of a turtle.jpg

In [ ]:
from google.colab.patches import cv2_imshow
def obtener_palabras_clave(nombre_imagen):
    nombre_sin_extension = os.path.splitext(os.path.basename(nombre_imagen))[0]
    indice_N = nombre_sin_extension.find('D')

    # Obtener las palabras clave después de 'N_'
    if indice_N != -1:
        palabras_clave = nombre_sin_extension[indice_N + 2:].split(' ')
        palabras_clave = [palabra for palabra in palabras_clave if not (palabra.isdigit() or palabra.lower() == 'pidinet1' or (palabra.lower() == 'pidinet2') or (palabra.isdigit() and 1 <= int(palabra) <= 5))]
    else:
        palabras_clave = []
    return palabras_clave

#------------------------

carpetasTortugas = os.listdir('/content/8Img/')
ruta_carpetasTortugas = [os.path.join('/content/8Img/', carpetaTortugas) for carpetaTortugas in carpetasTortugas]

carpetasTortugasRef = os.listdir('/content/8Img_Ref/')
ruta_carpetasTortugasRef = [os.path.join("/content/8Img_Ref/", carpetaTortugasRef) for carpetaTortugasRef in carpetasTortugasRef]

# Recorrer las rutas y comparar elementos dentro de las carpetas
for ruta1 in ruta_carpetasTortugas:
    tercer_elemento_ruta1 = ruta1.split('/')[3]  # El índice 3 corresponde al cuarto elemento (tercera carpeta)

    for ruta2 in ruta_carpetasTortugasRef:
        tercer_elemento_ruta2 = ruta2.split('/')[3]  # El índice 3 corresponde al cuarto elemento (tercera carpeta)

        # Comparar los terceros elementos
        if tercer_elemento_ruta1 == tercer_elemento_ruta2:
            print(f"Las rutas coinciden en la tercera carpeta: {ruta1} y {ruta2}")

            # Obtener listas de nombres de archivos en ambas carpetas

            carpetas1 = os.listdir(ruta1)
            ruta_carpetas1= [os.path.join(ruta1, carpeta1) for carpeta1 in carpetas1]
            carpetas2 = os.listdir(ruta2)
            ruta_carpetas2= [os.path.join(ruta2, carpeta2) for carpeta2 in carpetas2]
            print(ruta_carpetas1)

            for carpeta1 in ruta_carpetas1:
                imagenes1 = os.listdir(carpeta1)
                ruta_Imagenes1= [os.path.join(carpeta1, imagen) for imagen in imagenes1]
                for imagen in  ruta_Imagenes1:

                        palabras_clave_imagen_1 = obtener_palabras_clave(imagen)

                for carpeta2 in ruta_carpetas2:
                  print(carpeta1)
                  c1 = carpeta1.split('/')[4]
                  c2 = carpeta2.split('/')[4]

                  if c1 != c2:
                      imagenes2 = os.listdir(carpeta2)
                      ruta_Imagenes2= [os.path.join(carpeta2, img) for img in imagenes2]
                      for imagen_2 in  ruta_Imagenes2:

                            palabras_clave_imagen_2 = obtener_palabras_clave(imagen_2)
                            coincidencias = set(palabras_clave_imagen_1) & set(palabras_clave_imagen_2)
                            if len(coincidencias) >= 1:
                                      imagen1 = cv2.imread(imagen)
                                      imagen2 = cv2.imread(imagen_2)
                                      print(f"Coincidencias encontradas en palabras clave para imágenes:")

                                      print(f"Palabras clave coincidentes: {coincidencias}")
                                      print(imagen)
                                      print(imagen_2)

                                      # Definir los parámetros del ORB
                                      nfeatures = 10000  # Número máximo de características a retener
                                      scaleFactor = 1.1  # Factor de escala entre niveles de pirámide
                                      nlevels = 8  # Número de niveles en la pirámide
                                      edgeThreshold = 20  # Umbral para la supresión de bordes

                                      # Crear el objeto ORB con los parámetros especificados
                                      orb = cv2.ORB_create(nfeatures=nfeatures, scaleFactor=scaleFactor, nlevels=nlevels, edgeThreshold=edgeThreshold)

                                      keypoints1, descriptors1 = orb.detectAndCompute(imagen1, None)
                                      keypoints2, descriptors2 = orb.detectAndCompute(imagen2, None)

                                      img_orb1 = cv2.drawKeypoints(imagen1, keypoints1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                                      img_orb2 = cv2.drawKeypoints(imagen2, keypoints2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                                      img_orb1 = cv2.resize(img_orb1, (img_orb2.shape[1], img_orb2.shape[0]))

                                      frame = cv2.hconcat((img_orb1,img_orb2))


                                      bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                                      matches = bf.match(descriptors1, descriptors2)
                                      matches = sorted(matches, key=lambda x: x.distance)

                                      img_matches = cv2.drawMatches(imagen1, keypoints1, imagen2, keypoints2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

                                      good_matches = []
                                      for m in matches:
                                          pts1 = keypoints1[m.queryIdx].pt
                                          pts2 = keypoints2[m.trainIdx].pt
                                          good_matches.append(m)

                                      pts1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                                      pts2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

                                        # Inicializar una lista para almacenar el número de coincidencias en cada iteración de RANSAC
                                      num_matches_per_iteration = []
                                      numIteraciones=500
                                      for iteration in range(numIteraciones):  # Número de iteraciones en RANSAC (ajusta según sea necesario)
                                          # Encontrar homografía usando RANSAC
                                          M, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, ransacReprojThreshold=5.0, maxIters=iteration + 1)

                                          # Obtener la máscara de inliers
                                          matchesMask = mask.ravel().tolist()

                                          # Filtrar correspondencias basadas en la máscara RANSAC
                                          filtered_matches = [good_matches[i] for i in range(len(matchesMask)) if matchesMask[i] == 1]

                                          # Almacenar el número de coincidencias en esta iteración
                                          num_matches_per_iteration.append(len(filtered_matches))

                                        #   Dibujar las correspondencias filtradas (opcional, puedes comentar esta línea si no quieres visualizar cada iteración)
                                        #  img_matches = cv2.drawMatches(imagen1, keypoints1, imagen2, keypoints2, filtered_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                                        #  plt.figure(figsize=(10, 5))
                                        #  plt.imshow(img_matches)
                                        #  plt.title(f'Correspondencias Filtradas con Homografía RANSAC - Iteración {iteration + 1}')
                                        #  plt.show()
                                      porcentaje_aciertos = (len(filtered_matches) / min(len(keypoints1), len(keypoints2))) * 100
                                      # Dibujar las correspondencias filtradas
                                      img_matches = cv2.drawMatches(imagen1, keypoints1, imagen2, keypoints2, filtered_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                                      # Dibujar las mejores coincidencias en la imagen
                                      cv2_imshow(img_matches)
                                      import matplotlib.pyplot as plt

                                      # Graficar el número de coincidencias en cada iteración de RANSAC
                                      plt.figure(figsize=(10, 5))
                                      plt.plot(range(1, numIteraciones+1), num_matches_per_iteration, marker='o')
                                      plt.title('Número de Coincidencias en Cada Iteración de RANSAC')
                                      plt.xlabel('Iteración de RANSAC')
                                      plt.ylabel('Número de Coincidencias')
                                      plt.grid(True)
                                      plt.show()

                                      print(f"Porcentaje de coincidencia: {porcentaje_aciertos:.2f}%")



In [ ]:
import os

carpetaBase = '/content/Tortugas'
carpetasTortugas = os.listdir(carpetaBase)
for carpetaTortugas in carpetasTortugas:



          rutaCarpetasTortugas = os.path.join(carpetaBase, carpetaTortugas)

          carpetasImg = os.listdir(rutaCarpetasTortugas)
          for i, carpeta in enumerate(carpetasImg):
              if i <= 1:
                  ruta_elemento = os.path.join(rutaCarpetasTortugas, carpeta)
                  rutaOutput = '/content/outputPidinet/' + carpetaTortugas + '/' + carpeta
                  print(ruta_elemento)
                  !python /content/pidinet/main.py --model pidinet_converted --config carv4 --sa --dil -j 4 --gpu 0 --savedir {rutaOutput} --datadir {ruta_elemento} --dataset Custom --evaluate /content/pidinet/trained_models/table5_pidinet.pth --evaluate-converted




/content/Tortugas/8/20231206_071132
Namespace(savedir='/content/outputPidinet/8/20231206_071132', datadir='/content/Tortugas/8/20231206_071132', only_bsds=False, ablation=False, dataset=['Custom'], model='pidinet_converted', sa=True, dil=True, config='carv4', seed=1706236071, gpu='0', checkinfo=False, epochs=20, iter_size=24, lr=0.005, lr_type='multistep', lr_steps=None, opt='adam', wd=0.0001, workers=4, eta=0.3, lmbda=1.1, resume=False, print_freq=10, save_freq=1, evaluate='/content/pidinet/trained_models/table5_pidinet.pth', evaluate_converted=True, use_cuda=True)
{'layer0': 'cd', 'layer1': 'ad', 'layer2': 'rd', 'layer3': 'cv', 'layer4': 'cd', 'layer5': 'ad', 'layer6': 'rd', 'layer7': 'cv', 'layer8': 'cd', 'layer9': 'ad', 'layer10': 'rd', 'layer11': 'cv', 'layer12': 'cd', 'layer13': 'ad', 'layer14': 'rd', 'layer15': 'cv'}
initialization done
conv weights: lr 0.005000, wd 0.000100	bn weights: lr 0.005000, wd 0.000010	relu weights: lr 0.005000, wd 0.000000
cuda is used, with 1 gpu devi

**NOTE:** Before we save our `Detections` in Pascal VOC XML format, let's take a peek at the annotations we obtained. This step is optional, feel free to skip it.

In [ ]:
import cv2
import numpy as np

# Función para detectar formas en una máscara
def detectar_formas(mask):
    # Aplicar la transformada de Hough para detectar líneas
    lines = cv2.HoughLinesP(mask, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)

    # Si no se detectaron líneas, intenta detectar círculos
    if lines is None:
        circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=30, minRadius=0, maxRadius=0)
        if circles is not None:
            return "Círculo"

    return "Línea" if lines is not None else "Ninguna forma detectada"

# Cargar las máscaras (imágenes en blanco y negro)


# Cargar la imagen
image1 = cv2.imread('/content/Tortugas_Clip_Clasificacion/6/20231206_070634/D shell 0.51 C top shell 0.28.jpeg')
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
image2 = cv2.imread('/content/Tortugas_Clip_Clasificacion/6/20231206_070634/D head 0.52 C left shell 0.42.jpeg')
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
# Binarizar la imagen (convertir los píxeles diferentes de blanco en negro)
_, binary_image1 = cv2.threshold(gray_image1, 1, 255, cv2.THRESH_BINARY)
_, binary_image2 = cv2.threshold(gray_image2, 1, 255, cv2.THRESH_BINARY)



mask1 = binary_image1
mask2 = binary_image2

# Detectar formas en cada máscara
formas_mask1 = detectar_formas(mask1)
formas_mask2 = detectar_formas(mask2)

# Comparar las formas detectadas
if formas_mask1 == formas_mask2:
    print("Las máscaras tienen formas similares.")
else:
    print("Las máscaras tienen formas diferentes.")


In [ ]:
plot_images = []
plot_titles = []

box_annotator = sv.BoxAnnotator()
mask_annotator = sv.MaskAnnotator()

for image_name, detections in annotations.items():
    image = images[image_name]
    plot_images.append(image)
    plot_titles.append(image_name)

    labels = [
        f"{CLASSES[class_id]} {confidence:0.2f}"
        for _, _, confidence, class_id, _
        in detections]
    annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)
    annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
    plot_images.append(annotated_image)
    title = " ".join(set([
        CLASSES[class_id]
        for class_id
        in detections.class_id
    ]))
    plot_titles.append(title)

sv.plot_images_grid(
    images=plot_images,
    titles=plot_titles,
    grid_size=(len(annotations), 2),
    size=(2 * 4, len(annotations) * 4)
)

### Save labels in Pascal VOC XML

Before uploading our annotations to Roboflow, we must first save them to our hard drive. To do this, we will use one of the latest `supervision` features (available with the `0.6.0` update 🔥) - [dataset save](https://roboflow.github.io/supervision/dataset/core/).

In [ ]:
ANNOTATIONS_DIRECTORY = os.path.join(HOME, 'annotations')

MIN_IMAGE_AREA_PERCENTAGE = 0.002
MAX_IMAGE_AREA_PERCENTAGE = 0.80
APPROXIMATION_PERCENTAGE = 0.75

In [ ]:
sv.Dataset(
    classes=CLASSES,
    images=images,
    annotations=annotations
).as_pascal_voc(
    annotations_directory_path=ANNOTATIONS_DIRECTORY,
    min_image_area_percentage=MIN_IMAGE_AREA_PERCENTAGE,
    max_image_area_percentage=MAX_IMAGE_AREA_PERCENTAGE,
    approximation_percentage=APPROXIMATION_PERCENTAGE
)

### Upload annotations to Roboflow

Now we are ready to upload our annotations to Roboflow using the API.

In [ ]:
PROJECT_NAME = "auto-generated-dataset-7"
PROJECT_DESCRIPTION = "auto-generated-dataset-7"

In [ ]:
import roboflow
from roboflow import Roboflow

roboflow.login()

workspace = Roboflow().workspace()
new_project = workspace.create_project(
    project_name=PROJECT_NAME,
    project_license="MIT",
    project_type="instance-segmentation",
    annotation=PROJECT_DESCRIPTION)

In [ ]:
import os

for image_path in tqdm(image_paths):
    image_name = image_path.name
    annotation_name = f"{image_path.stem}.xml"
    image_path = str(image_path)
    annotation_path = os.path.join(ANNOTATIONS_DIRECTORY, annotation_name)
    new_project.upload(
        image_path=image_path,
        annotation_path=annotation_path,
        split="train",
        is_prediction=True,
        overwrite=True,
        tag_names=["auto-annotated-with-grounded-sam"],
        batch_name="auto-annotated-with-grounded-sam"
    )

## Convert Object Detection to Instance Segmentation Dataset

If you already have your dataset but it is annotated with bounding boxes, you can quickly and easily convert it to instance segmentation one. SAM allows using boxes as prompts. You can learn more about it from [this](https://blog.roboflow.com/how-to-use-segment-anything-model-sam/) tutorial.

### Download Object Detection Dataset from Roboflow

To use the dataset in this tutorial make sure to download it in [Pascal VOC XML](https://roboflow.com/formats/pascal-voc-xml) format. We will use [BlueBerries dataset](https://universe.roboflow.com/inkyu-sa-e0c78/blueberries-u0e84) as example.

In [ ]:
%cd {HOME}

import roboflow
from roboflow import Roboflow

roboflow.login()

rf = Roboflow()

project = rf.workspace("inkyu-sa-e0c78").project("blueberries-u0e84")
dataset = project.version(1).download("voc")

In [ ]:
dataset.location

In [ ]:
!ls {dataset.location}

### Load and Visualize Object Detection Dataset with Supervision

Learn more about Supervision Dataset support from our [documentation](https://roboflow.github.io/supervision/dataset/core/).

In [ ]:
object_detection_dataset = sv.Dataset.from_pascal_voc(
    images_directory_path=f"{dataset.location}/train",
    annotations_directory_path=f"{dataset.location}/train"
)

In [ ]:
import random
random.seed(9001)

**NOTE:** Rerun the cell below 👇 to take a look at different image from the dataset.

In [ ]:
image_names = list(object_detection_dataset.images.keys())
image_name = random.choice(image_names)

image = object_detection_dataset.images[image_name]
detections = object_detection_dataset.annotations[image_name]

box_annotator = sv.BoxAnnotator()

annotated_image = box_annotator.annotate(scene=image.copy(), detections=detections, skip_label=True)

%matplotlib inline
sv.plot_image(annotated_image, (16, 16))

### Run SAM convert Boxes into Masks

In [ ]:
from tqdm.notebook import tqdm

for image_name, image in tqdm(object_detection_dataset.images.items()):
    detections = object_detection_dataset.annotations[image_name]
    detections.mask = segment(
        sam_predictor=sam_predictor,
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB),
        xyxy=detections.xyxy
    )

**NOTE:** Rerun the cell below 👇 to take a look at different image from converted dataset.

In [ ]:
image_names = list(object_detection_dataset.images.keys())
image_name = random.choice(image_names)

image = object_detection_dataset.images[image_name]
detections = object_detection_dataset.annotations[image_name]

mask_annotator = sv.MaskAnnotator()

annotated_image = mask_annotator.annotate(scene=image.copy(), detections=detections)

%matplotlib inline
sv.plot_image(annotated_image, (16, 16))

In [ ]:
import shutil

# Ruta de la carpeta que deseas eliminar
ruta_carpeta = '/ruta/a/tu/carpeta'

# Elimina la carpeta y su contenido
shutil.rmtree(ruta_carpeta)


## Next steps

- If you liked the tutorial please leave ⭐ in our [supervision](https://github.com/roboflow/supervision) and [notebooks](https://github.com/roboflow/notebooks) repos.
- Review your dataset in Roboflow, correct any errors Grounding DINO / SAM made and [train real-time model](https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/).
- [SAM is built into Roboflow annotation tool as well](https://youtu.be/NAL2xcmMJiQ). If you need to refine the masks you can do it interactively in the browser.

## 🏆 Congratulations

### Learning Resources

Roboflow has produced many resources that you may find interesting as you advance your knowledge of computer vision:

- [Roboflow Notebooks](https://github.com/roboflow/notebooks): A repository of over 20 notebooks that walk through how to train custom models with a range of model types, from YOLOv7 to SegFormer.
- [Roboflow YouTube](https://www.youtube.com/c/Roboflow): Our library of videos featuring deep dives into the latest in computer vision, detailed tutorials that accompany our notebooks, and more.
- [Roboflow Discuss](https://discuss.roboflow.com/): Have a question about how to do something on Roboflow? Ask your question on our discussion forum.
- [Roboflow Models](https://roboflow.com): Learn about state-of-the-art models and their performance. Find links and tutorials to guide your learning.

### Convert data formats

Roboflow provides free utilities to convert data between dozens of popular computer vision formats. Check out [Roboflow Formats](https://roboflow.com/formats) to find tutorials on how to convert data between formats in a few clicks.

### Connect computer vision to your project logic

[Roboflow Templates](https://roboflow.com/templates) is a public gallery of code snippets that you can use to connect computer vision to your project logic. Code snippets range from sending emails after inference to measuring object distance between detections.